In [1]:
import numpy as np
from Bio import Phylo
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
import io 

#### Given an array of form:  
$[[parent_k,child_k,branchlength_k\ for\ k\ pairs\ in\ tree_i]\ for\ i\ trees]$  
1. Transform branch lengths to proportions of tree depth

In [2]:
def tree_depth(tree):
    '''given a tree in newick format, return overall depth'''
    
    tree = Phylo.read(io.StringIO(tree), 'newick')
    max_depth = max(tree.depths().values())
    
    return max_depth

def branch_ratio(data, tree_data, index):
    '''given a data_row with form [[parent ,child, branch_length]],
    return data_row with ratios of branch lengths'''
    
    tree = tree_data[tree_data['dreamID'] == index]['ground'].item()
    max_depth = tree_depth(tree)
    
    transformed_data = data #initialize
    for i in range(len(data)):
        transformed_data[i][2] = data[i][2] / max_depth
    
    return transformed_data
    

2. Convert Parent,Child pairs to 20x1 mutation array

In [43]:
def trit_det(parent, child):
    '''Given two trits from parent node 1 and child node 2 joined by an edge,
    return list alpha,beta determining mutation'''
    
    if parent == '1':
        if child == '2':#1->2
            alpha,beta = 0,1
        else:#1->0
            alpha,beta = 1,0
    else:#no mutation
        alpha,beta = 0,0
    
    return [alpha,beta]

def barcode_det(parent, child):
    '''Given two barcodes from parent node 1 and child node 2 joined by an edge,
    return 10x2 array with rows alpha_i beta_i'''
    
    parent = str(parent)
    child = str(child)
    alpha_beta_array = np.zeros((10,2))
    
    for i in range(10):
        alpha_beta_array[i, :] = trit_det(parent[i], child[i])
    
    return alpha_beta_array

def convert_pair(tree):
    '''Given a data row of form [[parent,child,branch_length]]
    return row of form [1x20 mutations, branch_length]'''
    
    converted_tree = []#initialize
    for pair in tree:
        mut_array = barcode_det(pair[0], pair[1]).reshape((20, ))
        converted_pair = [mut_array, pair[2]]
        converted_tree.append(converted_pair)
    
    return converted_tree

3. Reformat data for the model

In [44]:
def load_from_txt(data):
    '''Given a .txt file of the data, return an array'''
    loaded_data = []
    
    with open(data) as infile:
        lines = infile.readlines()
        for line in lines:
            line_list = line[2:len(line)-4].split('], ')
            new_line_list = []
            for pair in line_list:#[parent,child,branchlength] objects
                pair = pair[1:].split(', ')
                new_pair = pair
                new_pair[2] = float(pair[2])
                new_line_list.append(new_pair)
            loaded_data.append(new_line_list)
            
    return np.array(loaded_data)
            

DREAM_data = pd.read_csv('Data/DREAM_data_intMEMOIR.csv', sep = '\t')
DREAM_train = DREAM_data[30:]
DREAM_test = DREAM_data[:30]

load_from_txt('Data/testingDataFinalOutput.txt')
test_data_raw = load_from_txt('Data/testingDataFinalOutput.txt')
train_data_raw = load_from_txt('Data/trainingDataFinalOutput.txt')

reformat_train = []
reformat_test = []

reformat_pair = [reformat_train, reformat_test]
DREAM_pair = [DREAM_train, DREAM_test]
data_pair = [train_data_raw, test_data_raw]


for k in range(2): #0 = train, 1 = test
    for i in range(1, data_pair[k].shape[0]):
        tree_row = data_pair[k][i]
        tree_row = branch_ratio(tree_row, DREAM_pair[k], i)
        reformat_row = convert_pair(tree_row)
        reformat_pair[k].append(np.array(reformat_row))


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]


4. Setup data for the NN:  
    a. Train: will be split into minibatches by tree (list of DataLoader objects)  
    b. Test: Will be a Nx2 array with columns (mutation array, branchlength)

In [45]:
loader_list = []

for i in range(len(reformat_train)):
    tree_i = np.array(reformat_train[i])
    tree_i.shape = (len(reformat_train[i]), 2) # num_pairs x 2 array, columns are 1x20 mutation set and branch length
    
    train_input, train_label = np.vstack(tree_i[:, 0]), np.array(tree_i[:, 1], dtype = np.float32)
    input_train_torch = torch.from_numpy(train_input).type(torch.FloatTensor).view(-1, 1, 20)
    label_train_torch = torch.from_numpy(train_label).type(torch.FloatTensor)
    
    train_data = TensorDataset(input_train_torch, label_train_torch)
    loader_list.append(DataLoader(train_data, batch_size = 1))#within tree minibatch, steps of stride 1
    

test_input = []
test_label = []
#extract all pair_data into train:test columns, no minibatching
for i in range(len(reformat_test)):
    for pair_data in reformat_test[i]:
        test_input.append(pair_data[0])
        test_label.append(pair_data[1])
n_test = len(test_input)
input_test_torch = torch.from_numpy(np.array(test_input)).type(torch.FloatTensor).view(-1, 1, 20)
label_test_torch = torch.from_numpy(np.array(test_label)).type(torch.LongTensor)

In [46]:
class model(nn.Module):
    '''1 layer (FC) Neural Network'''
    
    def __init__(self):
        '''Define model module'''
        super(model, self).__init__()
        self.fc1 = nn.Linear(20, 20)
        self.fc2 = nn.Linear(20, 1)
    def forward(self, x):
        '''Define model activation'''
        x = self.fc1(x)
        return torch.sigmoid(self.fc2(x))

BRANCH_MODEL = model()
print(BRANCH_MODEL)
#define the optimizer
optimizer = optim.Adam(BRANCH_MODEL.parameters(), lr = 0.001)

model(
  (fc1): Linear(in_features=20, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=1, bias=True)
)


In [49]:
EPOCHS = 50
train_epoch_loss = []

for epoch in range(EPOCHS+1):
    train_loss = []
    
    #here train_set is the dataset of trees
    #each index correlates with set of [alpha_beta_array,branchlength]
    #at this point branch length needs to be normalized
    #for given tree 
    for train_loader in loader_list: #this would correlate with a DataLoader object

        for batch_index, (train_data, train_label) in enumerate(train_loader):
            
            BRANCH_MODEL.train()
            train_label_predicted = BRANCH_MODEL(train_data)
            
            #compute the loss
            loss = F.smooth_l1_loss(train_label_predicted, train_label)
            train_loss.append(loss.cpu().data.item())
            
            #reset the gradient
            optimizer.zero_grad()
            #backpropagate the loss
            loss.backward()
            #update the parameters
            optimizer.step()
        
        train_epoch_loss.append(np.mean(train_loss))
        
    if epoch%5 == 0:
        print("Epoch: {} | train_loss: {}".format(epoch, train_epoch_loss[-1]))
        

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Epoch: 0 | train_loss: 0.019174585720043612
Epoch: 5 | train_loss: 0.017670825847447717
Epoch: 10 | train_loss: 0.01762279914090963
Epoch: 15 | train_loss: 0.017600005481361005
Epoch: 20 | train_loss: 0.017580380410755866
Epoch: 25 | train_loss: 0.01755839718801089
Epoch: 30 | train_loss: 0.01754656458970984
Epoch: 35 | train_loss: 0.017546304859520766
Epoch: 40 | train_loss: 0.01754753333829103
Epoch: 45 | train_loss: 0.01754518019654988
Epoch: 50 | train_loss: 0.017541207681271632


In [50]:
#### Testing ####
test_data = TensorDataset(input_test_torch, label_test_torch)
test_loader = DataLoader(test_data, batch_size = n_test)
for i, (test_input, test_label) in enumerate(test_loader):
    BRANCH_MODEL.eval()
    predicted_label = BRANCH_MODEL(test_input)
    test_error = F.mse_loss(predicted_label, test_label)
    
print('Overall Error: {:.3f}'.format(test_error.mean()))

Overall Error: 0.081


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Using a target size (torch.Size([386])) that is different to the input size (torch.Size([386, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  import sys
